In [3]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn')
import seaborn as sns
import numpy as np
from scipy.stats import norm, skew
from sklearn.preprocessing import StandardScaler
from scipy import stats
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax

In [30]:
x_train = pd.read_csv('train_mod.csv')
x_test = pd.read_csv('test_mod.csv')
y_train = pd.read_csv('y_train.csv')

In [23]:
from sklearn.model_selection import GridSearchCV

# Linear models

In [111]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet
ridge = Ridge()
lasso = Lasso()
ElNet = ElasticNet()

In [112]:
# Ridge
para_ridge = {"alpha": np.logspace(-4,5,200)}
gs_ridge = GridSearchCV(ridge, para_ridge, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
gs_ridge.fit(x_train, y_train)
gs_ridge.best_params_

{'alpha': 6.905513520162331}

In [114]:
ridge.set_params(alpha = gs_ridge.best_params_['alpha'])
ridge.fit(x_train, y_train)
ridge.score(x_train, y_train)

0.9457476686666607

In [115]:
type(y_train.values)

numpy.ndarray

In [116]:
(np.sum((y_train.values - ridge.predict(x_train))**2)/len(y_train))**0.5

0.09306969753270902

In [117]:
y_train.values - ridge.predict(x_train)

array([[ 0.00768276],
       [-0.0460713 ],
       [ 0.03169271],
       ...,
       [ 0.00967619],
       [-0.01572397],
       [-0.00332653]])

In [118]:
# Lasso
para_lasso = {"alpha": np.logspace(-4,5,200)}
gs_lasso = GridSearchCV(lasso, para_lasso, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
gs_lasso.fit(x_train, y_train)
gs_lasso.best_params_

{'alpha': 0.0003872038781812557}

In [119]:
lasso.set_params(alpha = gs_lasso.best_params_['alpha'])
lasso.fit(x_train, y_train)
lasso.score(x_train, y_train)

0.9419755424505944

In [130]:
(np.sum((y_train.values - lasso.predict(x_train).reshape(-1,1))**2)/len(y_train))**0.5

0.09625086569730437

In [131]:
# ElasticNet
para_ElNet = {"alpha": np.logspace(-4,5,150),"l1_ratio": np.linspace(0,1,100)}
gs_ElNet = GridSearchCV(ElNet, para_ElNet, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
gs_ElNet.fit(x_train, y_train)
gs_ElNet.best_params_

{'alpha': 0.00040181564806038083, 'l1_ratio': 0.98989898989899}

In [132]:
ElNet.set_params(alpha = gs_ElNet.best_params_['alpha'],l1_ratio=gs_ElNet.best_params_['l1_ratio'])
ElNet.fit(x_train, y_train)
ElNet.score(x_train, y_train)

0.9417513661311411

In [133]:
(np.sum((y_train.values - ElNet.predict(x_train).reshape(-1,1))**2)/len(y_train))**0.5

0.09643661809511618